# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

In [22]:
# Access maps with unique API key
gmaps.comfigure(api_key=g_key)

NameError: name 'gmaps' is not defined

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
df = pd.read_csv("../output_data/cities.csv")
df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# get the data
data = df.loc[:, ["latitude", "longitude"]]

# create Base, init the graph 
figure_layout = {'width' : '800px', 'height': '600px','border' : '0px',
                'padding': '1px', 'margir': 'o auto 0 auto'}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

# show the plot
fig

KeyError: "None of [Index(['latitude', 'longitude'], dtype='object')] are in the [columns]"

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
mask = (df.wind_speed < 10) & (df.temperature < 80) & (df.temperature > 70) & (df.weather == "Clear")
hotel_df = df.loc[mask].reset_index(drop=True)
hotel_df

AttributeError: 'DataFrame' object has no attribute 'wind_speed'

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotels = []

for index, row in hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude
    
    # geocoordinates
    target_coordites = f"{lat}, {lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"
    
    # set up a parameters dictionary
    params = { "location": target_coordites, "keyword":  target_search, "radius": target_radius, "type": target_type, "key": g_key}
    
    # base url
    base_url = "https://maps.googleapis.com/api/place/nearbysearch/json"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if (len(data["results"])> 0):
        hotel = data["results"][0]["name"]
        address = data["results"][0]["vicinity"]
        rating = data["results"][0]["rating"]
        user_ratings_total = data["results"][0]["user_ratings_total"]
        
        else:
            hotel = None
            address = None
            rating = np.nan
            user_ratings_total = np.nan
            
            hotel = {"hotel": hotel,
                    "address": address, "rating": rating, "user_rating_total": user_rating_total}
            
            
            hotels.append(hotel)
    

SyntaxError: invalid syntax (4261398411.py, line 29)

In [32]:
df2 = pd.DataFrame(hotels)
df2.head()

NameError: name 'hotels' is not defined

In [8]:
lat = hotel_df.latitude[0]
lng = hotel_df.longitude[0]

NameError: name 'hotel_df' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
